### Boiler Plate

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics, linear_model

In [2]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from keras.utils import to_categorical

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [3]:
bin_train = np.genfromtxt('data/bindigit_trn.csv',delimiter=',',)
target_train = np.genfromtxt('data/targetdigit_trn.csv',delimiter=',',)
bin_test = np.genfromtxt('data/bindigit_tst.csv',delimiter=',',)
target_test = np.genfromtxt('data/targetdigit_tst.csv',delimiter=',',)

### DBN

In [4]:
def report_DBN(rbms = [150,100,50], iterations = 50, batch_size = 64, learning_rate = 0.01, verbose = True):
    #Initialize pipe
    networks = []
    count = 1
    for hidden_dim in rbms:
        rbm = BernoulliRBM(n_components = hidden_dim, batch_size = batch_size, learning_rate = learning_rate,
                      n_iter = iterations, verbose = verbose, random_state = 0)
        networks.append(('rbm{}'.format(count),rbm))
        count += 1

    networks.append(('mlp',MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',alpha=0.01)))
    model = Pipeline(networks)
    
    #Train model
    model.fit(bin_train, target_train)

    #Report results
    print(metrics.classification_report(
        target_test,
        model.predict(bin_test)))

In [5]:
report_DBN(rbms = [], iterations = 100, verbose = False)

             precision    recall  f1-score   support

        0.0       0.97      0.98      0.97       196
        1.0       0.96      0.97      0.96       227
        2.0       0.95      0.92      0.93       206
        3.0       0.95      0.92      0.94       202
        4.0       0.95      0.96      0.96       196
        5.0       0.91      0.91      0.91       182
        6.0       0.94      0.96      0.95       191
        7.0       0.95      0.92      0.94       205
        8.0       0.93      0.94      0.94       194
        9.0       0.93      0.96      0.94       201

avg / total       0.95      0.95      0.95      2000



### Stacked Autoencoders

In [6]:
def report_stacked_autoencoders(autoencoders, iterations = 50, batch_size=64 , verbose = True, target_train=target_train):
    input_layer = Input(shape=(784,))
    layer = input_layer
    #encode
    for hidden_dim in autoencoders:
        layer = Dense(hidden_dim, activation = 'relu')(layer)
    autoencoders.pop(-1)
    #decode
    for hidden_dim in reversed(autoencoders):
        layer = Dense(hidden_dim, activation = 'relu')(layer)
    layer = Dense(784, activation = 'relu')(layer)
    out = Dense(10, activation='sigmoid')(layer)    

    target_train = to_categorical(target_train, num_classes=10)
    model = Model(input_layer, out)
    model.compile(optimizer = 'adadelta', loss = 'mean_squared_error')
    model.fit(bin_train,target_train,
                   epochs=iterations,
                   batch_size=batch_size,
                   shuffle=True,
                   verbose=verbose)

    predictions = np.argmax(model.predict(bin_test),axis=1)
    print(metrics.classification_report(target_test,predictions))

In [7]:
report_stacked_autoencoders(autoencoders = [150,100,50], iterations = 50)

Epoch 1/50
8000/8000 [==============================] - 2s 229us/step - loss: 0.0963
Epoch 2/50
8000/8000 [==============================] - 1s 114us/step - loss: 0.0712
Epoch 3/50
8000/8000 [==============================] - 1s 113us/step - loss: 0.0428
Epoch 4/50
8000/8000 [==============================] - 1s 121us/step - loss: 0.0267
Epoch 5/50
8000/8000 [==============================] - 1s 127us/step - loss: 0.0190
Epoch 6/50
8000/8000 [==============================] - 1s 115us/step - loss: 0.0149
Epoch 7/50
8000/8000 [==============================] - 1s 112us/step - loss: 0.0121
Epoch 8/50
8000/8000 [==============================] - 1s 113us/step - loss: 0.0103
Epoch 9/50
8000/8000 [==============================] - 1s 116us/step - loss: 0.0090
Epoch 10/50
8000/8000 [==============================] - 1s 114us/step - loss: 0.0077
Epoch 11/50
8000/8000 [==============================] - 1s 118us/step - loss: 0.0068
Epoch 12/50
8000/8000 [==============================] - 1s 114

### Deeper MLP

In [8]:
def report_MLP(layers, iterations = 50, batch_size = 64, verbose = True, target_train = target_train):
    model = Sequential()
    for layer in layers:
        model.add(layer)

    target_train = to_categorical(target_train, num_classes=10)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    model.fit(bin_train,target_train,epochs=iterations,
              batch_size=batch_size,verbose=verbose, shuffle=False)
    predictions = np.argmax(model.predict(bin_test),axis=1)
    print(metrics.classification_report(target_test,predictions))

In [9]:
#https://keras.io/getting-started/sequential-model-guide/
layers = [
    Dense(100,input_shape=(784,), activation='relu'),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(10, activation='sigmoid'),
    ]

report_MLP(layers)

Epoch 1/50
8000/8000 [==============================] - 1s 174us/step - loss: 0.1032 - acc: 0.1771
Epoch 2/50
8000/8000 [==============================] - 1s 79us/step - loss: 0.0608 - acc: 0.5521
Epoch 3/50
8000/8000 [==============================] - 1s 73us/step - loss: 0.0364 - acc: 0.7674
Epoch 4/50
8000/8000 [==============================] - 1s 78us/step - loss: 0.0259 - acc: 0.8436
Epoch 5/50
8000/8000 [==============================] - 1s 77us/step - loss: 0.0212 - acc: 0.8705
Epoch 6/50
8000/8000 [==============================] - 1s 80us/step - loss: 0.0180 - acc: 0.8932
Epoch 7/50
8000/8000 [==============================] - 1s 80us/step - loss: 0.0159 - acc: 0.9036
Epoch 8/50
8000/8000 [==============================] - 1s 77us/step - loss: 0.0145 - acc: 0.9120
Epoch 9/50
8000/8000 [==============================] - 1s 78us/step - loss: 0.0132 - acc: 0.9195
Epoch 10/50
8000/8000 [==============================] - 1s 80us/step - loss: 0.0122 - acc: 0.9250
Epoch 11/50
8000/8